# Analyse van AskDelphi Relations & Tags endpoint
Dit notebook ondersteunt het verkennen, analyseren en valideren van de Relations & Tags functionaliteit binnen AskDelphi.  
De module biedt helpermethoden voor het toevoegen, beheren en inspecteren van relaties en tags die aan AskDelphi‑topics gekoppeld zijn.  
Het doel van dit notebook is om:

- inzicht te krijgen in de datastructuren en API‑interacties rondom relations & tags
- voorbeelddata te analyseren en patronen te ontdekken
- helpermethoden te testen en gedrag te evalueren
- experimentele of onderzoeksgerichte analyses uit te voeren ter verbetering van de contentstructuur in AskDelphi

Dit notebook vormt daarmee een flexibel startpunt voor verdere exploratie of debugging van de Relations & Tags endpoint.

### Importeren benodigde Python Libs

In [42]:
import dotenv
import logging
import sys
import os
import uuid
from pprint import pprint
from pathlib import Path
from src.askdelphi_client import AskDelphiClient

ROOT = Path().resolve().parent
sys.path.append(str(ROOT))
env_path = ROOT / ".env"

dotenv.load_dotenv(env_path)

logging.disable(sys.maxsize)

os.getenv("ASKDELPHI_PORTAL_CODE")

'S2RBZV-755ZA4'

### Connectie met AskDelphi opzetten

In [48]:
client = AskDelphiClient()
client.authenticate()

True

### Helper functies (@Todo vervangen door API functionaliteit)

In [ ]:
def get_topic_types() -> dict:
    """ Haalt de beschikbare AskDelphi-topictype IDs op.
    Returns:        
    dict: Mapping met topictype action, task en digitale coach procespagina"""
    topic_type_map = {}
    try:
        design = client.get_content_design()

        topic_types = design.get("topicTypes", [])
        # print(f"Found {len(topic_types)} topic types:\n")

        for i, tt in enumerate(topic_types[:100]):  # Show first 100
            if tt.get("title").lower() in ["action", "task", "digitale coach procespagina"]:
                # print(f"  {i+1}. {tt.get('title', 'Unknown')}")
                # print(f"     ID: {tt.get('key', 'N/A')}")
                # print(f"     Namespace: {tt.get('namespace', 'N/A')}")
                # print()

                topic_type_map[tt.get("title")] = tt.get("key")

        if len(topic_types) > 100:
            print(f"  ... and {len(topic_types) - 100} more")

    except Exception as e:
        print(f"FAILED: {e}")

    # print(f"topic type map:\n")
    # print(topic_type_map)

    return topic_type_map

def add_topic(topic_id: str, topicTitle: str, topicTypeId: str):
    """Voeg een topic toe van meegegeven topic type."""
    endpoint = "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic"
    return client._request(
        "POST",
        endpoint,
        json_data={
            "topicId": topic_id,
            "topicTitle": topicTitle,
            "topicTypeId": topicTypeId
        }
    )

def delete_topic(topicId: str, topicVersionId: str):
    """Verwijder een topic."""
    endpoint = f"v3/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topicId}/topicVersion/{topicVersionId}"
    client._request("DELETE", endpoint,
        json_data={
            "workflowActions": {}
        }
    )

def get_topicVersionId(topicId):
    endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topicId}/workflowstate"
    result = client._request("POST", endpoint,
                json_data={
                    "action": 1
                })
    return result["topicVersionId"]

### Uitwerking API client code

In [70]:
def add_relation(self, topic_id: str, topic_version_id: str, relation_type_id: str, target_topic_ids: list):
    """Voeg een relatie toe van dit topic naar andere topics."""
    endpoint = "v2/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic/{topic_id}/topicVersion/{topic_version_id}/relation"
    return self.session.post(
        endpoint,
        json={
            "relationTypeId": [3],
            "sourceTopicId": topic_id,
            "targetTopicIds": target_topic_ids,
        }
    )

def add_topic_with_relation(client: AskDelphiClient, topicId: str, topicTitle: str, topicTypeId: str, parentTopicId: str, parentTopicRelationTypeId: str, parentTopicVersionId: str):
    """Voeg een topic met een relatie naar andere topic toe.  
    Args:        
    - topicId (str): ID van het doel-topic.        
    - topicTitle (str): Titel van het doel-topic.        
    - topicTypeId (str): ID van het topictype.        
    - parentTopicId (str): ID van het bron-topic.
    - parentTopicRelationTypeId (str): ID van het relatietype.        
    - parentTopicVersionId (str): Versie-ID van het bron-topic."""
    endpoint = "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic"
    return client._request(
        "POST",
        endpoint,
        json_data={
            "topicId": topicId,
            "topicTitle": topicTitle,
            "topicTypeId": topicTypeId,
            "parentTopicId": parentTopicId,
            "parentTopicRelationTypeId": parentTopicRelationTypeId,
            "parentTopicVersionId": parentTopicVersionId
        }
    )

def add_tag(self, topic_id: str, topic_version_id: str, hierarchy_topic_id: str, hierarchy_node_id: str):
    """Voeg een tag toe aan een topic."""
    endpoint = f"v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id}/topicVersion/{topic_version_id}/tag"
    return self.session.post(
        endpoint,
        json={
            "tags": [
                {
                    "hierarchyTopicId": hierarchy_topic_id,
                    "hierarchyNodeId": hierarchy_node_id,
                }
            ]
        },
    )

def get_task_relation_id(topic_id: str, topicVersionId: str) -> str:
    """Get relationship ID for task"""
    task_relation_id = ""
    endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id}/topicVersion/{topicVersionId}/allowedrelations"
    result = client._request(
        "GET", 
        endpoint,
        json_data={}
    )
    for relation in result["topicAllowedRelations"]:
        if relation["topicTypeName"] == "Taak":
            task_relation_id = relation["relationTypeId"]
            break

    return task_relation_id


### Creatie Digicoach plus onderliggende taak

In [87]:
# Constant
DIGICOACH_NAME = "Digicoach test arie 4feb"
TASK_NAME = "Taak test 4feb"

# Get topic types
topic_type_map = get_topic_types()

# Create Digicoach topic
topic_id_digicoach = str(uuid.uuid4())
topic_type_id_dcpp = topic_type_map["Digitale Coach Procespagina"]
add_topic(topic_id_digicoach, DIGICOACH_NAME, topic_type_id_dcpp)

# Get Digicoach topic_version_id, task_relation_id
topic_version_id = get_topicVersionId(topic_id_digicoach)
task_relation_id = get_task_relation_id(topic_id_digicoach, topic_version_id)
print(f"Created Digicoach topic : {topic_id_digicoach}")

# Create Task 1 topic
topic_id_task_1 = str(uuid.uuid4())
add_topic_with_relation(
    client, 
    topic_id_task_1, 
    TASK_NAME, 
    topic_type_map["Task"], 
    topic_id_digicoach, 
    task_relation_id, 
    topic_version_id)

print(f"Created Task 1 topic : {topic_id_task_1}")

# Delete Task 1 topic
topic_version_id_task_1 = get_topicVersionId(topic_id_task_1)
delete_topic(topic_id_task_1, topic_version_id_task_1)
print(f"Deleted Task 1 topic : {topic_id_task_1}")

# Delete Digicoach topic
delete_topic(topic_id_digicoach, topic_version_id)
print(f"Deleted Digicoach topic : {topic_id_digicoach}")

Created Digicoach topic : 7a011faf-f6de-4c58-9072-16db4c7aaa05
Created Task 1 topic : 543005ba-06c9-472b-9477-8879844a98db
Deleted Task 1 topic : 543005ba-06c9-472b-9477-8879844a98db
Deleted Digicoach topic : 7a011faf-f6de-4c58-9072-16db4c7aaa05
